# Entraînement PhysAE étape par étape
Ce notebook remplace les anciens workflows et propose un canevas clair pour lancer l'entraînement des stages A, B1 et B2.
- Modifiez les dictionnaires pour personnaliser les hyperparamètres.
- Les paramètres disponibles sont listés ci-dessous pour les données et pour chaque stage.
- Toutes les commandes peuvent être exécutées directement telles quelles ou adaptées à vos besoins.

In [ ]:
import pandas as pd
from physae.parameter_catalog import (    TRAINER_PARAMETER_INFO,
    describe_parameters,
    list_data_parameters,
    list_stage_parameters,
)

## Paramètres des données
Chaque entrée peut être surchargée via le dictionnaire `data_overrides`.

In [ ]:
pd.DataFrame(describe_parameters(list_data_parameters()))

## Choix du stage et paramètres associés
Définissez le stage à entraîner (`"A"`, `"B1"` ou `"B2"`) puis inspectez ses paramètres.

In [ ]:
stage = "A"  # Modifier en "B1" ou "B2" selon le besoin
pd.DataFrame(describe_parameters(list_stage_parameters(stage)))

## Définition des overrides
Modifiez les dictionnaires ci-dessous pour adapter les hyperparamètres données/stage et les options du Trainer Lightning.
Les clés suivent la notation pointée (exemple : `model.encoder.params.width_mult`).

In [ ]:
data_overrides = {
    # "batch_size": 32,
    # "noise.train.std_add_range": [0.0, 0.02],
}

stage_overrides = {
    # "epochs": 25,
    # "base_lr": 2e-4,
    # "model.encoder.params.width_mult": 1.2,
}

trainer_kwargs = {
    "accelerator": "auto",
    "devices": "auto",
    # "precision": 16,
}

TRAINER_PARAMETER_INFO

## Lancer l'entraînement pour un stage unique
La cellule ci-dessous lance l'entraînement du stage sélectionné avec les overrides définis. Les métriques retournées et le chemin du dernier checkpoint sont affichés.

In [ ]:
from physae.simple_workflows import StageRunConfig, run_single_stage

single_stage_config = StageRunConfig(
    stage=stage,
    data_overrides=data_overrides,
    stage_overrides=stage_overrides,
    trainer_kwargs=trainer_kwargs,
    ckpt_dir="checkpoints_single_stage",
)
metrics, last_ckpt = run_single_stage(single_stage_config)
metrics, last_ckpt

## Chaîner plusieurs stages (A → B1 → B2)
Utilisez la configuration suivante pour entraîner plusieurs stages consécutivement. Vous pouvez définir des overrides spécifiques à chaque stage dans `sequence_stage_overrides`.
Si vous souhaitez uniquement affiner le stage B2 à partir d'un checkpoint existant, activez `fine_tune_only=True`.

In [ ]:
from physae.simple_workflows import StageSequenceConfig, run_stage_sequence

sequence_stage_overrides = {
    "A": {
        # "epochs": 20,
    },
    "B1": {
        # "refine_steps": 2,
    },
    "B2": {
        # "delta_scale": 0.1,
    },
}

sequence_config = StageSequenceConfig(
    stages=["A", "B"],  # "B" regroupe automatiquement B1 et B2
    data_overrides=data_overrides,
    stage_overrides=sequence_stage_overrides,
    trainer_kwargs=trainer_kwargs,
    ckpt_dir="checkpoints_sequence",
    fine_tune_only=False,
)
sequence_result = run_stage_sequence(sequence_config)
sequence_result.metrics, sequence_result.last_checkpoint

## Fine-tuning du stage B2 uniquement
Pour relancer un affinement sur B2 depuis les derniers checkpoints, utilisez la configuration suivante (assurez-vous que `ckpt_dir` contient déjà un checkpoint `stage_B2.ckpt`).

In [ ]:
fine_tune_config = StageSequenceConfig(
    stages=["B"],
    data_overrides=data_overrides,
    stage_overrides={
        "B2": {
            # "epochs": 10,
            # "ckpt_in": "checkpoints_sequence/stage_B2.ckpt",
        }
    },
    trainer_kwargs=trainer_kwargs,
    ckpt_dir="checkpoints_finetune",
    fine_tune_only=True,
)
fine_tune_result = run_stage_sequence(fine_tune_config)
fine_tune_result.metrics, fine_tune_result.last_checkpoint